ZH


Spark RDD feladatok

In [94]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
sc = SparkContext.getOrCreate()

In [95]:
x = "MWűGkyösdwinko Qau fSJpDaTrykv!w"

rdd = sc.parallelize([x[i:i+2] for i in range(0, len(x), 2)])
res = rdd.map(lambda x: x[0]).collect()

print("".join(res))

Működik a Spark!


Feladat 1. (2 pont)

A weboldalak.txt weboldalakat és azok szöveges tartalmát tartalmazza. Minden sor egy weboldal
címével kezdődik, aztán pedig a tartalma következik. A felhasználónk az "ELTE" kifejezésre keres rá,
nekünk pedig vissza kell adnunk számára a legrelevánsabb találatot. Határozod meg, hogy a
weboldalak.txt-ben található weboldalak közül melyikben szerepel legtöbbször a keresett kifejezés.
Csak azokat a weboldalakat vedd figyelembe, amelyek tartalma több, mint 10 szóból áll.
A programod írja ki azt a weboldalt, amelyekikben a legtöbbször szerepel a keresett kifejezés. Add
meg továbbá a keresett kifejezés előfordulásainak a számát és a weboldal tartalmában lévő szavak
számát is.
Egy lehetséges kimenet: ('wikipedia.hu', 10, 99), amelynek jelentése, hogy a wikipedia.hu weboldal
tartalmában a keresett kifejezés 10-szer szerepel és összesen 99 szóból áll.



In [79]:
data = sc.textFile('weboldalak.txt')\
.map(lambda l: l.upper())\
.map(lambda l: (l.split()[0], l[l.index(l.split()[0])+len(l.split()[0]):].count('ELTE'), len(l.split())-1))\
.filter(lambda x: x[2] > 10)\
.max(key= lambda x: x[1])\

data

('ELTE.HU', 6, 52)

Feladat 2. (2 pont)

A sentiment_input.txt minden sora egy-egy szöveges értékelése a Tenet című filmnek (forrás:
imdb.com). A feladat célja megszámolni, hány darab jó, rossz és semleges értékelés található a
fájlban.
• Egy értékelés akkor jó, ha a szövegben több pozitív szó található, mint negatív.
• Egy értékelés akkor rossz, ha több a negatív szó, mint a pozitív.
• Egy értékelés akkor semleges, ha a pozitív és negatív szavak száma megegyezik.
A pozitív és negatív szavak listája:
• Pozitív szavak: awesome, great, masterpiece, enjoyable
• Negatív szavak: worst, bad, boring, loud
Egy lehetséges kimenet: ('good', 10), ('bad', 9), ('neutral', 4). A bemeneti fájlban 23 sor (azaz
értékelés) található, ezért a pozitív, negatív és semleges értékelések összegének 23-at kell adnia.

In [99]:

positive = ['awesome', 'great', 'masterpiece', 'enjoyable']
negative = ['worst', 'bad', 'boring', 'loud']

def grade(inp):
  goodw = [1 for w in inp.split() if w in positive]
  badw = [1 for w in inp.split() if w in negative]


  if goodw > badw:
    return "good"
  elif badw > goodw:
    return "bad"
  else:
    return "neutral"



data = sc.textFile('sentiment_input.txt')\
.map(lambda l: l.replace(':',"").replace(",","").replace(".", "").replace("!","").replace('"',"").lower())\
.map(lambda l: grade(l))\
.countByValue()

print(f"(good, {data['good']}), (bad, {data['bad']}), (neutral, {data['neutral']})")

(good, 7), (bad, 5), (neutral, 11)


Dataframe feladatok

Az alábbi 4 feladat megoldásához a canvason található books.csv és orders.csv fájlokat kell beolvasni.
A fájlok egy könyveket árusító webshop adatait tartalmazza (forrás: kaggle.com). A books.csv minden
sora egy könyv adatait tartalmazza, az orders.csv minden sora pedig egy adott vásárlási folyamat egy
adott könyvhöz tartozó interakcióit adja meg (kattintások száma, kosárba helyezett darabszám,
megvásárolt darabszám).

books.csv oszlopai:
• rowID – a sor egyedi azonosítója
• itemID – egy könyv egyedi azonosítója
• title – a könyv címe
• author – a könyv szerzője
• pupblisher – a könyv kiadója

orders.csv oszlopai
• rowID – a sor egyedi azonosítója
• sessionID – egy vásárlási folyamat azonosítója
• itemID – egy könyv egyedi azonosítója
• click – a könyv oldalára történő kattintások száma a vásárlási folyamat során
• basket – az adott könyv kosárba helyezett darabszáma a vásárlási folyamat során
• order – az adott könyvből megrendelt darabszám a vásárlási folyamat során



In [43]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

In [45]:
books = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("books.csv")

orders = spark.read.format("csv")\
.option('header', True)\
.option("inferschema", True)\
.load("orders.csv")

books.show()
orders.show()

+-----+------+--------------------+------------------+--------------------+
|rowID|itemID|               title|            author|           publisher|
+-----+------+--------------------+------------------+--------------------+
|    0| 21310|Princess Poppy: T...|Janey Louise Jones|Penguin Random Ho...|
|    1| 73018|Einfach zeichnen!...|     Wiebke Krabbe|Schwager und Stei...|
|    2| 19194|         Red Queen 1|  Victoria Aveyard|Orion Publishing ...|
|    3| 40250|Meine Kindergarte...|              NULL|    Ars Edition GmbH|
|    4| 46107|Mein großes Schab...| Elizabeth Golding|Edition Michael F...|
|    5| 34217|        Ewig geliebt|        J. R. Ward|   Heyne Taschenbuch|
|    6| 31436| Meine Sticker-Tiere|              NULL|    Ars Edition GmbH|
|    7| 14576|Unsterblich 01 - ...|      Julie Kagawa|   Heyne Taschenbuch|
|    8| 17731|Unsterblich 02 - ...|      Julie Kagawa|   Heyne Taschenbuch|
|    9| 58723|Pedro und die Bet...|     Ursula Hasler|dtv Verlagsgesell...|
|   10| 7312

Feladat 3. (1 pont)
Add meg azt a kiadót, amely a legtöbb könyvet kiadta.
Elvárt kimenet: kiadó név.

In [87]:
from pyspark.sql.functions import col

books\
.groupBy(col("publisher"))\
.agg(countDistinct(col("title")).alias("Cnt"))\
.orderBy(col("Cnt").desc())\
.limit(1)\
.select("publisher")\
.show()

+---------------+
|      publisher|
+---------------+
|Books on Demand|
+---------------+



Feladat 4. (1 pont)
A vásárlással végződő folyamatok során átlagosan hány darab könyvet vásároltak?
Elvárt kimenet: egy valós szám.

In [89]:
from pyspark.sql.functions import sum
orders\
.groupBy(col("sessionID"))\
.agg(sum(col("order")).alias("ocnt"))\
.where(col("ocnt") > 0)\
.agg(avg(col("ocnt")))\
.show()

+----------------+
|       avg(ocnt)|
+----------------+
|1.33439033597584|
+----------------+



Feladat 5. (1 pont)
Azon vásárlások közül, ahol a felhasználó már a kosárba helyezte a terméket, hány esetben vette meg
a kosárba helyezett termékeket és hány esetben nem? Add meg az esetek számát!
Elvárt kimenet: két szám (hány esetben vették meg a termékeket és hány esetben nem).

In [90]:
#all = orders.where(col("basket") > 0).count()

bought = orders\
.where((col("order") == col("basket")) & (col("basket") > 0))\
.count()

not_bought = orders\
.where((col("order") != col("basket")) & (col("basket") > 0))\
.count()

#print(bought + not_bought == all)
bought, not_bought

(4069, 40919)

Feladat 6. (1 pont)
Melyik könyvre kattintottak a legtöbbször? Add meg a könyv címét és a kattintások számát!
Elvárt kimenet: egy könyv címe és a kattintások száma.

In [78]:
from pyspark.sql.functions import sum
orders.alias("O")\
.groupBy(col("itemID"))\
.agg(sum(col("click")).alias("sumc"))\
.join(books.alias("B"), col("O.itemID") == col("B.itemID"), "inner")\
.select("title","sumc")\
.orderBy(col("sumc").desc())\
.limit(1)\
.show()

+--------------------+----+
|               title|sumc|
+--------------------+----+
|Harry Potter Box ...|3229|
+--------------------+----+

